In [65]:
import numpy as np
from shapely.geometry import Point
import pandas as pds
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry import Point
import pandas as pd
import sqlite3

In [54]:
lights = pd.read_csv('Lights.csv')
collision = pd.read_csv('NYPD_Motor_Vehicle_Collisions.csv')

In [60]:
lights.shape, collision.shape

((328121, 53), (792851, 29))

In [56]:
lights = lights.dropna(subset=['Latitude', 'Longitude'])
collision = collision.dropna(subset=['LATITUDE', 'LONGITUDE'])

# Remove unusable data from Lights

In [58]:
lights = lights[lights['Complaint Type'] == 'Street Light Condition']

In [59]:
descriptors_useful = ['Street Light Out', 
                      'Fixture/Luminaire Hanging', 
                      'Lamppost Knocked Down', 
                      'Fixture/Luminaire Damaged', 
                      'Lamppost Missing', 
                      'Lamppost Damaged',
                      'Street Light Lamp Missing', 
                      'Fixture/Luminaire Missing', 
                      'Street Light Lamp Dim',
                      'Flood Light Lamp Missing', 
                      'Wood Pole Knocked Down', 
                      'Wood Pole Leaning', 
                      'Wood Pole Missing']
lights = lights[lights['Descriptor'].isin(descriptors_useful)]

# Dates to DateTime

In [61]:
lights['CreatedDate'] = pd.to_datetime(lights['Created Date'], infer_datetime_format=True)
lights['ClosedDate'] = pd.to_datetime(lights['Closed Date'], infer_datetime_format=True)
collision['datetime'] = pd.to_datetime(collision['DATE'] + ' ' + collision['TIME'], infer_datetime_format=True)
lights = lights.drop(['Created Date', 'Closed Date'], 1)

# Merging Lights and Collision

In [62]:
bin_size = 0.001
binning = lambda x: round(x / bin_size) * bin_size
lights['lat_bin'] = lights['Latitude'].apply(binning)
lights['long_bin'] = lights['Longitude'].apply(binning)
collision['lat_bin'] = collision['LATITUDE'].apply(binning)
collision['long_bin'] = collision['LONGITUDE'].apply(binning)

In [73]:
lights['uniquekey'] = lights['Unique Key']
collision['uniquekey'] = collision['UNIQUE KEY']

In [74]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
conn.text_factory = str

#write the tables
collision.to_sql('collision', conn, index=False)
lights.to_sql('lights', conn, index=False)

In [93]:
query = '''
        select
            collision.uniquekey, count(lights.uniquekey)
        from
            collision left join lights
                on datetime between CreatedDate and ClosedDate 
                and collision.lat_bin = lights.lat_bin
                and collision.long_bin = lights.long_bin
        group by
            collision.uniquekey
        '''

df = pd.read_sql_query(query, conn)

In [95]:
df.describe()

,uniquekey,count(lights.uniquekey)
count,7.928510e+05,792851.000000
mean,1.945638e+06,0.049127
std,1.528130e+06,0.258530
min,2.200000e+01,0.000000
25%,2.216295e+05,0.000000
50%,3.033589e+06,0.000000
75%,3.342113e+06,0.000000
max,3.631696e+06,12.000000


In [97]:
lights.Descriptor.unique()

array(['Street Light Out', 'Fixture/Luminaire Damaged', 'Lamppost Missing',
       'Lamppost Damaged', 'Street Light Lamp Missing',
       'Fixture/Luminaire Hanging', 'Fixture/Luminaire Missing',
       'Lamppost Knocked Down', 'Street Light Lamp Dim',
       'Flood Light Lamp Missing', 'Wood Pole Knocked Down',
       'Wood Pole Leaning', 'Wood Pole Missing'], dtype=object)

In [99]:
df['Lights Complaints'] = df['count(lights.uniquekey)']
df = df.drop('count(lights.uniquekey)', 1)

In [ ]:
df.to_pickle('SaveName')